In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/qaly_input_full.csv")

In [3]:
to_fit

,Description,Value,Lower Bound,Upper Bound
0,Child-Ototoxicity,24.139307,22.391619,25.887412
1,Child-NCD,19.485947,16.284814,21.524365
2,Caregiver-Oto or NCD,11.137435,9.696120,12.316693
3,Caregiver-Infant Death,12.054636,10.482292,12.971836


In [4]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):

    scale_lower_bound = np.around((abs(lower_bound - value)/2)**2 / value, 2)
    scale_upper_bound = scale_lower_bound + 10
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    scale_lower_bound = int((((upper_bound - value)/1.8)**2 / value) / 100) * 100
    scale_upper_bound = scale_lower_bound + 100000
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [5]:
scale_lower_bound = 0.01
scale_upper_bound = 0.5
linspace_points = 0.01
search_range = np.arange(scale_lower_bound, scale_upper_bound, linspace_points)


In [6]:
search_range

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49])

In [7]:
to_fit["a"], to_fit["scale"] = np.vectorize(fit_encompassing_gamma_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

a: 603.48 	 scale: 0.04
a: 603.48 	 scale: 0.04
a: 129.91 	 scale: 0.15
a: 185.62 	 scale: 0.06
a: 200.91 	 scale: 0.06


In [8]:
to_fit["gamma_mean"] = to_fit["a"] * to_fit["scale"]
to_fit["gamma_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["a"], scale=to_fit["scale"])
to_fit["gamma_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["a"], scale=to_fit["scale"])

In [9]:
to_fit

,Description,Value,Lower Bound,Upper Bound,a,scale,gamma_mean,gamma_2.5pct,gamma_97.5pct
0,Child-Ototoxicity,24.139307,22.391619,25.887412,603.482677,0.04,24.139307,22.251539,26.102839
1,Child-NCD,19.485947,16.284814,21.524365,129.906311,0.15,19.485947,16.279354,22.976527
2,Caregiver-Oto or NCD,11.137435,9.696120,12.316693,185.623922,0.06,11.137435,9.592800,12.795684
3,Caregiver-Infant Death,12.054636,10.482292,12.971836,200.910597,0.06,12.054636,10.445306,13.777584


In [10]:
to_fit.to_csv("Data/qaly_output_full.csv")